In [ ]:
import math
import numpy as np
from copy import deepcopy, copy
import matplotlib.pyplot as plt
from numpy.lib.scimath import sqrt
from numpy import pi, exp, real, imag, linspace, sin, cos

In [ ]:
class Function():
  def __init__(self, coefs, args, func, op):
    self.coefs, self.args, self.func, self.op = coefs, args, func, op

In [ ]:
def bhaskara(a, b, c):
  if a == 0:
    return [-c/b]
  else:
    delta_squared = sqrt(b ** 2 - (4 * a * c))
    x1 = (-b + delta_squared) / (2 * a)
    x2 = (-b - delta_squared) / (2 * a)
    return [x1, x2]

In [ ]:
def cramer(A, R):
    l = A.shape[0]
    D = np.linalg.det(A)
    det = np.zeros(l)
    for column in range(l) :
        Acopy = deepcopy(A)
        for i in range(l):
            Acopy[i, column] = R[i]
        det[column] = np.linalg.det(Acopy)
    res = np.zeros(l)
    for i in range(len(det)) :
        res[i] = det[i]/D
    return list(res)

In [ ]:
def step(n):
  return 1 if n >= 0 else 0

In [ ]:
def get_natural(coefs):
  a, b, c = coefs
  if c == 0:
    a, b, c = c, a, b
  roots = bhaskara(a, b, c)
  func = lambda n: [root ** n for root in roots]
  return Function(None, roots, func, [EXPONENTIAL for root in roots])

In [ ]:
def get_particular(x):
  c_x, args, op = x

  if op == EXPONENTIAL:
    func = lambda n: [arg ** n for arg in args]
  else:
    func = lambda n: [cos(args[0] * n), sin(args[0] * n)]
  return Function(None, args, func, [op])

In [ ]:
def get_forced(y_n, y_p, b):
  args = y_n.args + y_p.args
  k = len(b) - 1
  func = lambda n: list(map(lambda y: (y * step(n - k)), y_n.func(n) + y_p.func(n)))
  op = y_n.op + y_p.op
  return Function(None, args, func, op)

In [ ]:
def get_x(x):
  c_x, args, op = x
  if op == EXPONENTIAL:
    func = lambda n: [c_x * arg ** n * step(n) for arg in args]
  elif op == COSSINE:
    func = lambda n: [c_x * cos(arg * n) * step(n) for arg in args]
  else:
    func = lambda n: [c_x * cos(arg * n) * step(n) for arg in args]
  return Function(c_x, args, func, op)

In [ ]:
def calculate_forced(a, y_f, b, x):
  total_coefs = len(y_f.args)
  mat = []
  R = []
  start = len(b) - 1
  for n in range(start, total_coefs + start):
    sigma = [0 for i in range(total_coefs)]
    for k in range(len(a)):
      result = y_f.func(n - k)
      temp = list(map(lambda y: a[k] * y, result))
      sigma = list(map(sum, zip(temp, sigma)))
    mat.append(sigma)
    sigma = 0
    for k in range(len(b)):
      result = x.func(n - k)
      temp = list(map(lambda x: b[k] * x, result))
      sigma = sigma + sum(temp)
    R.append(sigma)
  mat = np.array(mat)
  R = np.array(R)
  return cramer(mat, R)

In [ ]:
def calculate_natural(y_n, y_init):
  mat = []
  R = []
  for n, val in y_init:
    mat.append(y_n.func(n))
    R.append(val)
  mat = np.array(mat)
  R = np.array(R)
  return cramer(mat, R)

In [ ]:
def get_complete(y_n, y_f):
  func = lambda n: y_n.func(n) + y_f.func(n)
  args = y_n.args + y_f.args
  coefs = y_n.coefs + y_f.coefs
  op = y_n.op + y_f.op
  return Function(coefs, args, func, op)

In [ ]:
def loop_print(text, y):
  aux_print = {EXPONENTIAL: "{0} * ({1})^n", COSSINE: "{0} * cos({1} * n)", SINE: "{0} * sin({1} * n)"}
  for i in range(len(y.coefs)):
    string = aux_print[y.op[i]]
    text = text + string.format(round(y.coefs[i], 3), round(y.args[i], 3)) + " + "
  return text

In [ ]:
def print_complete_answer(y_n, y_f, b):
  text = "Resposta completa: "
  text = loop_print(text, y_n) + "("
  text = loop_print(text, y_f)[:-3] + ")" + (" * u[n]" if len(b) == 1 else " * u[n - {0}]".format(len(b) - 1))
  return print(text)

# Célula de resolução de equações a diferenças:

<a href="https://www.codecogs.com/eqnedit.php?latex=\large&space;a_0\cdot&space;y[n]&plus;a_1\cdot&space;y[n-1]&plus;a_2\cdot&space;y[n-2]=&space;\sum_{i=0}^{N}&space;b_i\cdot&space;x[n&plus;k_i]" target="_blank"><img src="https://latex.codecogs.com/gif.latex?\large&space;a_0\cdot&space;y[n]&plus;a_1\cdot&space;y[n-1]&plus;a_2\cdot&space;y[n-2]=&space;\sum_{i=0}^{N}&space;b_i\cdot&space;x[n&plus;k_i]" title="\large a_0\cdot y[n]+a_1\cdot y[n-1]+a_2\cdot y[n-2]= \sum_{i=0}^{N} b_i\cdot x[n+k_i]" /></a>


Só testei o descobrimento de respostas completas. Para fim de exemplo, a letra B do exercício 2.22 do Haykin e a 1ª questão da 2ª parte da prova de Sinais e Sistemas 2 foram recriadas abaixo.

In [ ]:
EXPONENTIAL = 0
COSSINE = 1
SINE = 2

In [ ]:
# LETRA B DO EXERCÍCIO 2.22 DO HAYKIN:
# y[n] + 0 * y[n - 1] - (1/9) * y [n - 2] = 0 * x[n] + x[n - 1]
# ONDE x[n] = u[n]
# E y[-1] = 1, y[-2] = 0

# a0 = 1, a1 = 0 E a2 = -1/9
a = [1, 0, -1/9]

# b0 = 0 e b1 = 1
b = [0, 1]

#    1 * 1    ^       n  = QUE É UM ARMENGUE PARA DECLARAR u[n]
x = (1, [1], EXPONENTIAL)

# O 1º NÚMERO DO PAR É O n de y[n] e O 2º É O VALOR DAQUELA POSIÇÃO
y_init = [(-1, 1), (-2, 0)]

y_n = get_natural(a)
y_p = get_particular(x)
x = get_x(x)


y_f = get_forced(y_n, y_p, b)

y_n.coefs = calculate_natural(y_n, y_init)
y_f.coefs = calculate_forced(a, y_f, b, x)

print_complete_answer(y_n, y_f, b)

Resposta completa: 0.167 * (0.333)^n + -0.167 * (-0.333)^n + (-0.75 * (0.333)^n + -0.375 * (-0.333)^n + 1.125 * (1)^n) * u[n - 1]


In [ ]:
# RESOLUÇÃO DA 1º QUESTÃO DA 2ª PARTE DA PROVA DE SINAIS E SISTEMAS 2:

a = [1, -1/4, -1/8]
b = [1, 1]

#    1 * cos((1/5) * pi)
x = (1, [(1/5) * pi], COSSINE)

y_init = [(-1, 2), (-2, -1)]

y_n = get_natural(a)
y_p = get_particular(x)
x = get_x(x)


y_f = get_forced(y_n, y_p, b)


y_n.coefs = calculate_natural(y_n, y_init)
y_f.coefs = calculate_forced(a, y_f, b, x)

print_complete_answer(y_n, y_f, b)

Resposta completa: 0.583 * (0.5)^n + -0.208 * (-0.25)^n + (5.393 * (0.5)^n + 3.551 * (-0.25)^n + -0.0 * cos(0.628 * n)) * u[n - 1]
